In [ ]:
#!pip install -U langgraph langchain_openai tavily-python langchain_community langchain_elasticsearch

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_openai import ChatOpenAI
from langchain_elasticsearch import ElasticsearchStore
import os

In [ ]:
class State(TypedDict):
    # Messages have the type "list". The `add_messages` function
    # in the annotation defines how this state key should be updated
    # (in this case, it appends messages to the list, rather than overwriting them)
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

web_search = TavilySearchResults(max_results=2)
# tool.invoke("who is ashish tiwari from elastic?")

In [ ]:
def getWorkplaceData(query: str) -> str:
    
    """Use this to answer all internal organisation data. For example company policies, office, sales, team structure etc but not employee specific."""
    es = ElasticsearchStore(
        es_cloud_id=os.environ["ES_CLOUD_ID"],
        es_api_key=os.environ["ES_API_KEY"],
        index_name="workplace-data",
        strategy=ElasticsearchStore.SparseVectorRetrievalStrategy(
            model_id=".elser_model_2_linux-x86_64"
        )
    )

    docs = es.similarity_search(query, k=2)
    return "".join(doc.page_content for doc in docs)
    

In [ ]:
tools = [web_search, getWorkplaceData]
llm = ChatOpenAI(model="gpt-4o").bind_tools(tools)
llm_with_tools=llm.bind_tools(tools)

In [ ]:
def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)

In [ ]:
tool_node = ToolNode(tools=tools)
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
# Any time a tool is called, we return to the chatbot to decide the next step
graph_builder.add_edge("tools", "chatbot")
graph_builder.set_entry_point("chatbot")
graph = graph_builder.compile()

In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

In [ ]:
while True:
    user_input = input("\nUser: ")
    if user_input.lower() in ["quit", "exit", "q"]:
        print("Goodbye!")
        break
    for event in graph.stream({"messages": ("user", user_input)}):
        for value in event.values():
            print("\nAssistant:", value["messages"][-1].content)